In [33]:
from app.api.yazz import CityAppClient

from typing import Any

In [ ]:
# app/rag/service.py

from typing import Any

from app.api.yazz import CityAppClient


class CityAssistant:
    """
    Простой "городской помощник" поверх CityAppClient.
    Возвращает структурированный JSON-ответ (dict), а не строку.
    """

    def __init__(self, client: CityAppClient | None = None):
        self.client = client or CityAppClient()

    def answer(self, question: str) -> dict[str, Any]:
        """
        Главный метод: принимает вопрос, возвращает dict с полями:
        - scenario
        - question
        - summary
        - items
        - meta
        """
        q = question.lower()

        # 1. МФЦ
        if 'мфц' in q:
            return self._answer_mfc(question)

        # 2. Пенсионеры / "серебряный возраст"
        if 'пенсионер' in q or 'серебрян' in q or 'пожил':
            return self._answer_pensioner(question)

        # 3. Афиша / мероприятия
        if 'афиш' in q or 'меро' in q or 'концерт' in q:
            return self._answer_afisha(question)

        # 4. Поликлиники
        if 'поликлиник' in q or 'поликлиника' in q or 'больниц' in q:
            return self._answer_polyclinic(question)

        # 5. Школы / садики
        if 'школ' in q or 'лицей' in q or 'гимнази' in q or 'садик' in q or 'детсад' in q:
            return self._answer_education(question)

        return {
            'scenario': 'unknown',
            'question': question,
            'summary': (
                'Пока я умею помогать с вопросами про МФЦ, услуги для пенсионеров, '
                'афишу мероприятий, поликлиники и школы/детские сады.\n'
                'Попробуйте переформулировать вопрос, например: '
                '"Где ближайший МФЦ к адресу ...?" или '
                '"Какие кружки для пенсионеров есть в Невском районе?"'
            ),
            'items': [],
            'meta': {},
        }

    # ---------- приватные методы под конкретные сценарии ----------

    def _answer_mfc(self, question: str) -> dict[str, Any]:
        """
        МФЦ: пока считаем, что пользователь указал адрес прямо в вопросе.
        Например: "Где ближайший МФЦ к Невскому проспекту 1?"
        """
        data = self.client.find_nearest_mfc(question)

        if not data:
            return {
                'scenario': 'mfc',
                'question': question,
                'summary': 'Не удалось найти ближайший МФЦ по указанному адресу.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'address_query': question,
                },
            }

        # упрощаем до одного элемента в списке
        item = {
            'name': data.get('name'),
            'address': data.get('address'),
            'metro': data.get('metro'),
            'phones': data.get('phones'),
            'hours': data.get('hours'),
            'link': data.get('link'),
            'coords': data.get('coords'),
            'distance_km': data.get('distance_km'),
        }

        summary_lines: list[str] = [
            'Нашёл информацию о ближайшем МФЦ:',
            f"Название: {item['name']}",
            f"Адрес: {item['address']}",
        ]
        if item.get('metro'):
            summary_lines.append(f"Ближайшее метро: {item['metro']}")
        if item.get('hours'):
            summary_lines.append(f"График работы: {item['hours']}")

        summary = '\n'.join(summary_lines)

        return {
            'scenario': 'mfc',
            'question': question,
            'summary': summary,
            'items': [item],
            'meta': {
                'source': 'yazzh_api',
            },
        }

    def _answer_pensioner(self, question: str) -> dict[str, Any]:
        """
        Услуги для пенсионеров: MVP — фиксированный район и категория.
        Потом можно доставать это из текста вопроса.
        """
        district = 'Невский'
        categories = ['Здоровье']

        data = self.client.pensioner_services(district=district, category=categories, count=5)

        if not data:
            return {
                'scenario': 'pensioner',
                'question': question,
                'summary': f'Не удалось получить услуги для пенсионеров в районе {district}.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'district': district,
                    'categories': categories,
                },
            }

        if isinstance(data, dict):
            raw_items = data.get('data', [])
        else:
            raw_items = data

        items: list[dict[str, Any]] = []
        for item in raw_items[:5]:
            items.append(
                {
                    'name': item.get('name') or item.get('title'),
                    'address': item.get('address'),
                    'organization': item.get('organization'),
                    'schedule': item.get('schedule') or item.get('timetable'),
                    'category': item.get('category'),
                    'link': item.get('url') or item.get('link'),
                }
            )

        if not items:
            return {
                'scenario': 'pensioner',
                'question': question,
                'summary': f'В районе {district} не найдено подходящих услуг для пенсионеров.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'district': district,
                    'categories': categories,
                },
            }

        summary_lines: list[str] = [f'Некоторые услуги для пенсионеров в районе {district}:']
        for it in items:
            line = f"- {it['name']}"
            if it.get('address'):
                line += f" (адрес: {it['address']})"
            summary_lines.append(line)

        summary = '\n'.join(summary_lines)

        return {
            'scenario': 'pensioner',
            'question': question,
            'summary': summary,
            'items': items,
            'meta': {
                'source': 'yazzh_api',
                'district': district,
                'categories': categories,
            },
        }

    def _answer_afisha(self, question: str) -> dict[str, Any]:
        """
        Афиша: фиксированный период дат (MVP).
        Потом можно парсить даты из вопроса.
        """
        start = '2025-11-21T00:00:00'
        end = '2025-12-22T00:00:00'

        events = self.client.afisha_events(start_date=start, end_date=end)

        if not events:
            return {
                'scenario': 'afisha',
                'question': question,
                'summary': 'Не удалось получить афишу мероприятий на ближайшее время.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'start_date': start,
                    'end_date': end,
                },
            }

        if isinstance(events, dict):
            raw_items = events.get('data', [])
        else:
            raw_items = events

        items: list[dict[str, Any]] = []
        for ev in raw_items[:5]:
            items.append(
                {
                    'title': ev.get('name') or ev.get('title'),
                    'place': ev.get('place') or ev.get('venue'),
                    'datetime': ev.get('start_date') or ev.get('date'),
                    'price': ev.get('price'),
                    'is_free': ev.get('free'),
                    'link': ev.get('url') or ev.get('link'),
                }
            )

        if not items:
            return {
                'scenario': 'afisha',
                'question': question,
                'summary': 'На выбранный период мероприятий не найдено.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'start_date': start,
                    'end_date': end,
                },
            }

        summary_lines: list[str] = ['Некоторые мероприятия в Санкт-Петербурге:']
        for it in items:
            line = f"- {it['title']}"
            if it.get('datetime'):
                line += f" — {it['datetime']}"
            if it.get('place'):
                line += f" ({it['place']})"
            summary_lines.append(line)

        summary = '\n'.join(summary_lines)

        return {
            'scenario': 'afisha',
            'question': question,
            'summary': summary,
            'items': items,
            'meta': {
                'source': 'yazzh_api',
                'start_date': start,
                'end_date': end,
            },
        }

    def _answer_polyclinic(self, question: str) -> dict[str, Any]:
        data = self.client.get_polyclinics_by_address(question)

        if not data:
            return {
                'scenario': 'polyclinic',
                'question': question,
                'summary': 'По указанному адресу не удалось найти поликлиники.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'address_query': question,
                },
            }

        items: list[dict[str, Any]] = []
        for clinic in data[:5]:
            items.append(
                {
                    'name': clinic.get('name'),
                    'address': clinic.get('address'),
                    'phones': clinic.get('phones'),
                    'url': clinic.get('url'),
                }
            )

        summary_lines: list[str] = ['Поликлиники, связанные с этим адресом:']
        for it in items:
            line = f"- {it['name']}"
            if it.get('address'):
                line += f" (адрес: {it['address']})"
            summary_lines.append(line)

        summary = '\n'.join(summary_lines)

        return {
            'scenario': 'polyclinic',
            'question': question,
            'summary': summary,
            'items': items,
            'meta': {
                'source': 'yazzh_api',
                'address_query': question,
            },
        }

    def _answer_education(self, question: str) -> dict[str, Any]:
        district = 'Центральный'
        schools = self.client.get_schools_by_district(district)

        if not schools:
            return {
                'scenario': 'education',
                'question': question,
                'summary': f'В районе {district} не удалось найти школ в справочнике.',
                'items': [],
                'meta': {
                    'source': 'yazzh_api',
                    'district': district,
                },
            }

        items: list[dict[str, Any]] = []
        for s in schools[:5]:
            items.append(
                {
                    'name': s.get('name'),
                    'address': s.get('address'),
                    'phone': s.get('phone'),
                }
            )

        summary_lines: list[str] = [f'Примеры школ в районе {district}:']
        for it in items:
            line = f"- {it['name']}"
            if it.get('address'):
                line += f" (адрес: {it['address']})"
            summary_lines.append(line)

        summary = '\n'.join(summary_lines)

        return {
            'scenario': 'education',
            'question': question,
            'summary': summary,
            'items': items,
            'meta': {
                'source': 'yazzh_api',
                'district': district,
            },
        }


In [44]:
assistant = CityAssistant()

resp = assistant.answer('какие мероприятия для пенсионеров')
print(resp['scenario'])
print(resp['summary'])
print(resp['items'][0])


pensioner
Некоторые услуги для пенсионеров в районе Невский:
- Галотерапия (адрес: г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А)
- Оздоровительная физическая культура (адрес: г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А)
- Скандинавская ходьба (адрес: г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А)
- Соматическая гимнастика (адрес: г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А)
- Су-джок терапия  (адрес: г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А)
{'name': 'Галотерапия', 'address': 'г.Санкт-Петербург, Дальневосточный проспект, дом 8, корпус 1, литера А', 'organization': None, 'schedule': None, 'category': ['Здоровье'], 'link': None}
